In [1]:
import sys
from pathlib import Path

# Project root (one level ABOVE src)
PROJ_ROOT = Path(r"C:\Users\quantbase\Desktop\SyStrat_factmod")

if str(PROJ_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJ_ROOT))

print("Project root on sys.path:", PROJ_ROOT in map(Path, map(str, sys.path)))

from src.fm_config import DATA_INT
print("DATA_INT =", DATA_INT)


Project root on sys.path: True
DATA_INT = C:\Users\quantbase\Desktop\SyStrat_factmod\data_int


In [18]:
from datetime import date
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from src.fm_config import DATA_INT
from src.fm_data_prices import load_us_prices, load_nse_prices
from src.fm_fundamentals import (
    load_us_income_ttm,
    load_us_balance_ttm,
    load_us_cashflow_ttm,
)
from src.fm_returns import build_returns_all, build_mkt_cap_all
from src.fm_panel_us import build_us_panel
from src.fm_factors import (
    build_us_tier1_factors,
    build_us_tier2_factors,
    build_us_tier3_factors,
    build_style_us_full,
)
from src.fm_factor_panel import build_us_factor_panel
from src.fm_factor_model_us import estimate_us_factor_model
from src.fm_decompose_us import (
    _load_factor_panel_us,
    _load_factor_model_us,
    build_equal_weight_portfolio,
    decompose_portfolio_us,
    sanity_check_decomposition_us,
    _attach_sector_exposures_us,
)



In [3]:
from src.fm_config import (
    US_UNIVERSE_DEFAULT, US_BACKTEST_START, US_BACKTEST_END, DATA_INT
)

from src.fm_factors import (
    build_us_tier1_factors, build_us_tier2_factors,
    build_us_tier3_factors, build_style_us_full
)
from src.fm_factor_panel import build_us_factor_panel

In [4]:
US_TICKERS = ["NVDA","AAPL","MSFT","AMZN","GOOGL","GOOG","AVGO","META","TSLA","BRK.B",
 "LLY","JPM","WMT","ORCL","V","XOM","JNJ","MA","NFLX","ABBV",
 "COST","PLTR","BAC","AMD","PG",
 "HD","GE","CSCO","KO","CVX",
 "UNH","IBM","WFC","CAT","MS","MU","PM","AXP","TMUS","MRK",
 "GS","RTX","ABT","CRM","MCD","TMO","PEP","ISRG","LIN","AMAT"]
    #["AAPL", "MSFT", "AMZN", "GOOGL","NVDA","META"]  # change here only
START = date(2020, 10, 31)
END   = date(2025, 11, 20)

US_TICKERS, START, END

(['NVDA',
  'AAPL',
  'MSFT',
  'AMZN',
  'GOOGL',
  'GOOG',
  'AVGO',
  'META',
  'TSLA',
  'BRK.B',
  'LLY',
  'JPM',
  'WMT',
  'ORCL',
  'V',
  'XOM',
  'JNJ',
  'MA',
  'NFLX',
  'ABBV',
  'COST',
  'PLTR',
  'BAC',
  'AMD',
  'PG',
  'HD',
  'GE',
  'CSCO',
  'KO',
  'CVX',
  'UNH',
  'IBM',
  'WFC',
  'CAT',
  'MS',
  'MU',
  'PM',
  'AXP',
  'TMUS',
  'MRK',
  'GS',
  'RTX',
  'ABT',
  'CRM',
  'MCD',
  'TMO',
  'PEP',
  'ISRG',
  'LIN',
  'AMAT'],
 datetime.date(2020, 10, 31),
 datetime.date(2025, 11, 20))

In [5]:
# Prices + fundamentals (US only)
# 1) Prices (SimFin)
df_us_px = load_us_prices(
    tickers=US_TICKERS,
    start=START,
    end=END,
    save=True,          # overwrites prices_us.parquet/csv
)
df_us_px.tail()

Dataset "us-shareprices-daily" on disk (14 days old).
- Loading from disk ... 

c:\Users\quantbase\.conda\envs\factmod-311\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
[US] Saved 49,008 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\prices_us.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\prices_us.csv


,ticker,date,SimFinId,open,high,low,close,adj_close,volume,Dividend,shares_outstanding
49003,XOM,2024-11-15,121214,120.40,121.24,119.13,119.31,115.08,19051713,NaN,4.395095e+09
49004,XOM,2024-11-18,121214,119.79,120.62,119.27,120.31,116.04,14243701,NaN,4.395095e+09
49005,XOM,2024-11-19,121214,119.75,119.75,118.20,118.63,114.42,11584382,NaN,4.395095e+09
49006,XOM,2024-11-20,121214,119.17,120.47,118.64,120.32,116.05,11360932,NaN,4.395095e+09
49007,XOM,2024-11-21,121214,121.08,122.55,120.27,121.93,117.61,14675422,NaN,4.395095e+09


In [6]:
# 2) Fundamentals (income, balance, cashflow)
df_inc = load_us_income_ttm(tickers=US_TICKERS, save=True)
df_bal = load_us_balance_ttm(tickers=US_TICKERS, save=True)
df_cf  = load_us_cashflow_ttm(tickers=US_TICKERS, save=True)

Dataset "us-income-ttm" on disk (14 days old).
- Loading from disk ... Done!


c:\Users\quantbase\.conda\envs\factmod-311\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


[US][income_ttm] Saved 814 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\us_income_ttm.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\us_income_ttm.csv
Dataset "us-balance-ttm" on disk (14 days old).
- Loading from disk ... 

c:\Users\quantbase\.conda\envs\factmod-311\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
[US][balance_ttm] Saved 815 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\us_balance_ttm.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\us_balance_ttm.csv
Dataset "us-cashflow-ttm" on disk (14 days old).
- Loading from disk ... Done!
[US][cashflow_ttm] Saved 814 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\us_cashflow_ttm.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\us_cashflow_ttm.csv


c:\Users\quantbase\.conda\envs\factmod-311\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


In [7]:
df_inc.tail()

,ticker,report_date,SimFinId,Currency,Fiscal Year,Fiscal Period,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
809,XOM,2023-09-30,121214,USD,2023,Q3,2023-10-31,2024-11-04,4.117313e+09,4.118349e+09,...,-3.451600e+10,-784000000.0,6.100000e+10,NaN,61000000000,-1.860300e+10,42397000000,NaN,41130000000,41130000000
810,XOM,2023-12-31,121214,USD,2023,Q4,2024-02-28,2024-11-04,4.052000e+09,4.053036e+09,...,-2.347500e+10,-849000000.0,5.278300e+10,NaN,52783000000,-1.542900e+10,37354000000,NaN,36010000000,36010000000
811,XOM,2024-03-31,121214,USD,2024,Q1,2024-04-29,2025-05-05,4.026000e+09,4.027036e+09,...,-2.317800e+10,-911000000.0,4.834900e+10,NaN,48349000000,-1.427200e+10,34077000000,NaN,32800000000,32800000000
812,XOM,2024-06-30,121214,USD,2024,Q2,2024-08-05,2025-08-04,4.089786e+09,4.089786e+09,...,-2.185400e+10,-933000000.0,5.035800e+10,NaN,50358000000,-1.486300e+10,35495000000,NaN,34160000000,34160000000
813,XOM,2024-09-30,121214,USD,2024,Q3,2024-11-04,2025-11-03,4.197508e+09,4.197508e+09,...,-2.087100e+10,-971000000.0,4.968500e+10,NaN,49685000000,-1.456500e+10,35120000000,NaN,33700000000,33700000000


In [8]:
df_bal.tail()

,ticker,report_date,SimFinId,Currency,Fiscal Year,Fiscal Period,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity
810,XOM,2023-09-30,121214,USD,2023,Q3,2023-10-31,2023-10-31,4.117313e+09,4.118349e+09,...,4.743000e+09,7.118600e+10,3.651000e+10,9.354000e+10,1.647260e+11,1.616500e+10,-2.535120e+11,4.501380e+11,2.075330e+11,372259000000
811,XOM,2023-12-31,121214,USD,2023,Q4,2024-02-28,2025-02-19,4.052000e+09,4.053036e+09,...,4.090000e+09,6.531600e+10,3.748300e+10,9.846300e+10,1.637790e+11,1.778100e+10,-2.549170e+11,4.539270e+11,2.125380e+11,376317000000
812,XOM,2024-03-31,121214,USD,2024,Q1,2024-04-29,2024-04-29,4.026000e+09,4.027036e+09,...,8.227000e+09,7.192100e+10,3.221300e+10,9.294500e+10,1.648660e+11,1.797100e+10,-2.578910e+11,4.583390e+11,2.130520e+11,377918000000
813,XOM,2024-06-30,121214,USD,2024,Q2,2024-08-05,2024-08-05,4.089786e+09,4.089786e+09,...,6.621000e+09,7.076300e+10,3.656500e+10,1.136780e+11,1.844410e+11,4.678100e+10,-2.284830e+11,4.632940e+11,2.762660e+11,460707000000
814,XOM,2024-09-30,121214,USD,2024,Q3,2024-11-04,2024-11-04,4.197508e+09,4.197508e+09,...,5.632000e+09,6.999300e+10,3.691800e+10,1.155230e+11,1.855160e+11,4.693600e+10,-2.340490e+11,4.676640e+11,2.764000e+11,461916000000


In [9]:
df_cf.tail()

,ticker,report_date,SimFinId,Currency,Fiscal Year,Fiscal Period,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
809,XOM,2023-09-30,121214,USD,2023,Q3,2023-10-31,2024-08-05,4.117313e+09,4.118349e+09,...,59308000000,-2.147400e+10,4.391000e+09,0.0,-1.891500e+10,-1.486900e+10,-4.729000e+09,-1.776700e+10,-3.867900e+10,2509000000
810,XOM,2023-12-31,121214,USD,2023,Q4,2024-02-28,2024-11-04,4.052000e+09,4.053036e+09,...,55369000000,-2.191900e+10,4.078000e+09,0.0,-1.927400e+10,-1.494100e+10,-2.390000e+08,-1.774800e+10,-3.429700e+10,1903000000
811,XOM,2024-03-31,121214,USD,2024,Q1,2024-04-29,2025-05-05,4.026000e+09,4.027036e+09,...,53692000000,-2.158100e+10,3.927000e+09,0.0,-1.892600e+10,-1.501100e+10,-9.440000e+08,-1.641900e+10,-3.377200e+10,673000000
812,XOM,2024-06-30,121214,USD,2024,Q2,2024-08-05,2025-05-05,4.089786e+09,4.089786e+09,...,54869000000,-2.245700e+10,3.566000e+09,754000000.0,-1.942700e+10,-1.559500e+10,-3.727000e+09,-1.740500e+10,-3.816600e+10,-3069000000
813,XOM,2024-09-30,121214,USD,2024,Q3,2024-11-04,2025-08-04,4.197508e+09,4.197508e+09,...,56475000000,-2.369700e+10,2.776000e+09,754000000.0,-2.138800e+10,-1.617200e+10,-5.084000e+09,-1.850500e+10,-4.121300e+10,-6001000000


In [10]:
df_ret = build_returns_all(save=True)
df_ret.head()

[RETURNS] Saved 49,113 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\returns_all.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\returns_all.csv


,date,ticker,market,ret
0,2020-11-03,AAPL,US,0.015353
1,2020-11-04,AAPL,US,0.040837
2,2020-11-05,AAPL,US,0.035494
3,2020-11-06,AAPL,US,-0.002856
4,2020-11-09,AAPL,US,-0.019968


In [11]:
df_mkt = build_mkt_cap_all(save=True)
df_mkt.tail()

[MKT_CAP] Saved 49,163 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\mkt_cap_all.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\mkt_cap_all.csv


C:\Users\quantbase\Desktop\SyStrat_factmod\src\fm_returns.py:150: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(frames, ignore_index=True)


,date,ticker,market,mkt_cap
49158,2024-11-15,XOM,US,5.243787e+11
49159,2024-11-18,XOM,US,5.287738e+11
49160,2024-11-19,XOM,US,5.213901e+11
49161,2024-11-20,XOM,US,5.288178e+11
49162,2024-11-21,XOM,US,5.358939e+11


In [12]:
#----diagnostic----

In [13]:
import pandas as pd
from src.fm_config import DATA_INT

mkt = pd.read_parquet(DATA_INT / "mkt_cap_all.parquet")
print("mkt_cap_all tickers:", mkt["ticker"].unique())
print(
    mkt[mkt["market"] == "US"]
    .groupby("ticker")["mkt_cap"]
    .apply(lambda s: s.gt(0).mean())
)


mkt_cap_all tickers: ['AAPL' 'ABBV' 'ABT' 'AMAT' 'AMD' 'AMZN' 'AVGO' 'AXP' 'BAC' 'CAT' 'COST'
 'CRM' 'CSCO' 'CVX' 'GE' 'GOOG' 'GS' 'HD' 'IBM' 'ISRG' 'JNJ' 'JPM' 'KO'
 'LIN' 'LLY' 'MA' 'MCD' 'META' 'MRK' 'MS' 'MSFT' 'MU' 'NFLX' 'NVDA' 'ORCL'
 'PEP' 'PG' 'PLTR' 'PM' 'RELIANCE' 'RTX' 'SBIN' 'TMO' 'TMUS' 'TSLA' 'UNH'
 'V' 'WFC' 'WMT' 'XOM']
ticker
AAPL    1.0
ABBV    1.0
ABT     1.0
AMAT    1.0
AMD     1.0
AMZN    1.0
AVGO    1.0
AXP     1.0
BAC     1.0
CAT     1.0
COST    1.0
CRM     1.0
CSCO    1.0
CVX     1.0
GE      1.0
GOOG    1.0
GS      1.0
HD      1.0
IBM     1.0
ISRG    1.0
JNJ     1.0
JPM     1.0
KO      1.0
LIN     1.0
LLY     1.0
MA      1.0
MCD     1.0
META    1.0
MRK     1.0
MS      1.0
MSFT    1.0
MU      1.0
NFLX    1.0
NVDA    1.0
ORCL    1.0
PEP     1.0
PG      1.0
PLTR    1.0
PM      1.0
RTX     1.0
TMO     1.0
TMUS    1.0
TSLA    1.0
UNH     1.0
V       1.0
WFC     1.0
WMT     1.0
XOM     1.0
Name: mkt_cap, dtype: float64


In [14]:
#----End---

In [12]:
# US panel + style factors
panel_us = build_us_panel(
    tickers=US_TICKERS,
    start=START,
    end=END,
    save=True,
)
panel_us.head()

[US_PANEL] Saved 48,960 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\funds_panel_us.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\funds_panel_us.csv


,date,ticker,market,ret,mkt_cap,inc_report_date,inc_SimFinId,inc_Currency,inc_Fiscal Year,inc_Fiscal Period,...,cf_Change in Fixed Assets & Intangibles,cf_Net Change in Long Term Investment,cf_Net Cash from Acquisitions & Divestitures,cf_Net Cash from Investing Activities,cf_Dividends Paid,cf_Cash from (Repayment of) Debt,cf_Cash from (Repurchase of) Equity,cf_Net Cash from Financing Activities,cf_Net Change in Cash,inc_age_days
0,2020-11-03,AAPL,US,0.015353,1.877679e+12,2020-09-30,111052.0,USD,2020.0,Q4,...,-7.309000e+09,5.453000e+09,-1.524000e+09,-4.289000e+09,-1.408100e+10,2.499000e+09,-7.235800e+10,-8.682000e+10,-1.043500e+10,34.0
1,2020-11-04,AAPL,US,0.040837,1.954357e+12,2020-09-30,111052.0,USD,2020.0,Q4,...,-7.309000e+09,5.453000e+09,-1.524000e+09,-4.289000e+09,-1.408100e+10,2.499000e+09,-7.235800e+10,-8.682000e+10,-1.043500e+10,35.0
2,2020-11-05,AAPL,US,0.035494,2.023724e+12,2020-09-30,111052.0,USD,2020.0,Q4,...,-7.309000e+09,5.453000e+09,-1.524000e+09,-4.289000e+09,-1.408100e+10,2.499000e+09,-7.235800e+10,-8.682000e+10,-1.043500e+10,36.0
3,2020-11-06,AAPL,US,-0.002856,2.017944e+12,2020-09-30,111052.0,USD,2020.0,Q4,...,-7.309000e+09,5.453000e+09,-1.524000e+09,-4.289000e+09,-1.408100e+10,2.499000e+09,-7.235800e+10,-8.682000e+10,-1.043500e+10,37.0
4,2020-11-09,AAPL,US,-0.019968,1.977650e+12,2020-09-30,111052.0,USD,2020.0,Q4,...,-7.309000e+09,5.453000e+09,-1.524000e+09,-4.289000e+09,-1.408100e+10,2.499000e+09,-7.235800e+10,-8.682000e+10,-1.043500e+10,40.0


In [13]:
panel_us.tail()

,date,ticker,market,ret,mkt_cap,inc_report_date,inc_SimFinId,inc_Currency,inc_Fiscal Year,inc_Fiscal Period,...,cf_Change in Fixed Assets & Intangibles,cf_Net Change in Long Term Investment,cf_Net Cash from Acquisitions & Divestitures,cf_Net Cash from Investing Activities,cf_Dividends Paid,cf_Cash from (Repayment of) Debt,cf_Cash from (Repurchase of) Equity,cf_Net Cash from Financing Activities,cf_Net Change in Cash,inc_age_days
48955,2024-11-15,XOM,US,-0.010368,5.243787e+11,2024-09-30,121214.0,USD,2024.0,Q3,...,-2.369700e+10,2.776000e+09,754000000.0,-2.138800e+10,-1.617200e+10,-5.084000e+09,-1.850500e+10,-4.121300e+10,-6.001000e+09,46.0
48956,2024-11-18,XOM,US,0.008382,5.287738e+11,2024-09-30,121214.0,USD,2024.0,Q3,...,-2.369700e+10,2.776000e+09,754000000.0,-2.138800e+10,-1.617200e+10,-5.084000e+09,-1.850500e+10,-4.121300e+10,-6.001000e+09,49.0
48957,2024-11-19,XOM,US,-0.013964,5.213901e+11,2024-09-30,121214.0,USD,2024.0,Q3,...,-2.369700e+10,2.776000e+09,754000000.0,-2.138800e+10,-1.617200e+10,-5.084000e+09,-1.850500e+10,-4.121300e+10,-6.001000e+09,50.0
48958,2024-11-20,XOM,US,0.014246,5.288178e+11,2024-09-30,121214.0,USD,2024.0,Q3,...,-2.369700e+10,2.776000e+09,754000000.0,-2.138800e+10,-1.617200e+10,-5.084000e+09,-1.850500e+10,-4.121300e+10,-6.001000e+09,51.0
48959,2024-11-21,XOM,US,0.013381,5.358939e+11,2024-09-30,121214.0,USD,2024.0,Q3,...,-2.369700e+10,2.776000e+09,754000000.0,-2.138800e+10,-1.617200e+10,-5.084000e+09,-1.850500e+10,-4.121300e+10,-6.001000e+09,52.0


In [14]:
# Tiered styles
t1 = build_us_tier1_factors(save=True)
t2 = build_us_tier2_factors(save=True)
t3 = build_us_tier3_factors(save=True)
style_full = build_style_us_full(save=True)

style_full.tail()


[STYLE_US_T1] Saved 48,960 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\style_us_tier1.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\style_us_tier1.csv
[STYLE_US_T2] Saved 48,960 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\style_us_tier2.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\style_us_tier2.csv
[STYLE_US_T3] Saved 48,960 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\style_us_tier3.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\style_us_tier3.csv
[STYLE_US_FULL] Saved 48,960 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\style_us_full.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\style_us_full.csv


,date,ticker,market,SIZE,VAL,MOM,QUAL,ROE_z,GM_z,CashEY_z,LEV_raw,LEV_z,EMA20_GAP,VOL_Z,LIQ
48955,2024-11-15,XOM,US,0.351973,2.981858,-0.689703,0.597279,-0.417773,-1.062687,2.375030,0.401623,-0.816151,-0.093724,-0.456367,0.491498
48956,2024-11-18,XOM,US,0.357501,2.957847,-0.695034,0.582786,-0.417773,-1.062687,2.354239,0.401623,-0.816151,0.004029,-0.462297,0.432512
48957,2024-11-19,XOM,US,0.337181,2.989019,-0.741720,0.607253,-0.417773,-1.062687,2.391217,0.401623,-0.816151,-0.274193,-0.485213,0.341094
48958,2024-11-20,XOM,US,0.351328,2.965385,-0.712199,0.590732,-0.417773,-1.062687,2.368467,0.401623,-0.816151,-0.045949,-0.483705,0.370294
48959,2024-11-21,XOM,US,0.359192,2.958905,-0.687759,0.590567,-0.417773,-1.062687,2.368394,0.401623,-0.816151,0.048292,-0.492455,0.427053


In [15]:
factor_panel = build_us_factor_panel(save=True)

factor_panel.tail()

[FACTOR_PANEL_US] Saved 48,960 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_panel_us.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_panel_us.csv


,date,ticker,market,ret,mkt_cap,SIZE,VAL,MOM,QUAL,ROE_z,GM_z,CashEY_z,LEV_raw,LEV_z,EMA20_GAP,VOL_Z,LIQ
48955,2024-11-21,UNH,US,-0.005012,5.498607e+11,0.388591,-0.190104,-0.965951,-1.657380,-0.392150,-1.465691,-0.609711,0.651013,0.271262,-0.041688,-0.078218,-1.019331
48956,2024-11-21,V,US,0.008166,6.390493e+11,0.560349,-0.432355,-0.732878,-0.352234,0.225702,NaN,-0.571953,0.583964,-0.021091,0.266506,-0.532514,-0.376384
48957,2024-11-21,WFC,US,0.016988,2.491458e+11,-0.515954,NaN,0.496490,NaN,NaN,NaN,NaN,NaN,NaN,1.321984,0.838022,0.695784
48958,2024-11-21,WMT,US,0.013879,7.105010e+11,0.681456,0.077250,0.609728,-0.837241,-0.271600,-1.373447,0.401707,0.641362,0.229180,0.826325,-0.883185,0.636490
48959,2024-11-21,XOM,US,0.013381,5.358939e+11,0.359192,2.958905,-0.687759,0.590567,-0.417773,-1.062687,2.368394,0.401623,-0.816151,0.048292,-0.492455,0.427053


In [17]:
panel = pd.read_parquet(DATA_INT / "funds_panel_us.parquet")

print("funds_panel_us tickers:", panel["ticker"].unique())
print(panel.groupby("ticker")[["ret", "mkt_cap"]].count())

funds_panel_us tickers: ['AAPL' 'ABBV' 'ABT' 'AMAT' 'AMD' 'AMZN' 'AVGO' 'AXP' 'BAC' 'CAT' 'COST'
 'CRM' 'CSCO' 'CVX' 'GE' 'GOOG' 'GS' 'HD' 'IBM' 'ISRG' 'JNJ' 'JPM' 'KO'
 'LIN' 'LLY' 'MA' 'MCD' 'META' 'MRK' 'MS' 'MSFT' 'MU' 'NFLX' 'NVDA' 'ORCL'
 'PEP' 'PG' 'PLTR' 'PM' 'RTX' 'TMO' 'TMUS' 'TSLA' 'UNH' 'V' 'WFC' 'WMT'
 'XOM']
         ret  mkt_cap
ticker               
AAPL    1020     1020
ABBV    1020     1020
ABT     1020     1020
AMAT    1020     1020
AMD     1020     1020
AMZN    1020     1020
AVGO    1020     1020
AXP     1020     1020
BAC     1020     1020
CAT     1020     1020
COST    1020     1020
CRM     1020     1020
CSCO    1020     1020
CVX     1020     1020
GE      1020     1020
GOOG    1020     1020
GS      1020     1020
HD      1020     1020
IBM     1020     1020
ISRG    1020     1020
JNJ     1020     1020
JPM     1020     1020
KO      1020     1020
LIN     1020     1020
LLY     1020     1020
MA      1020     1020
MCD     1020     1020
META    1020     1020
MRK     1020    

In [17]:
%cd C:\Users\quantbase\Desktop\SyStrat_factmod
!python -m src.fm_sector_us

C:\Users\quantbase\Desktop\SyStrat_factmod
[SECTOR_US] Built sector dummies: 49 tickers, 9 sector columns.
[SECTOR_US] Saved to:
  C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\sector_dummies_us.parquet
  C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\sector_dummies_us.csv
[SECTOR_US] Head:
  ticker  ... sector_Materials
0   AAPL  ...            False
1   MSFT  ...            False
2   AMZN  ...            False
3   META  ...            False
4   NVDA  ...            False

[5 rows x 11 columns]


In [18]:
import importlib

In [19]:
import src.fm_factor_model_us
from src.fm_factor_model_us import estimate_us_factor_model

In [20]:
importlib.reload(src.fm_factor_model_us)


<module 'src.fm_factor_model_us' from 'C:\\Users\\quantbase\\Desktop\\SyStrat_factmod\\src\\fm_factor_model_us.py'>

In [27]:

out = estimate_us_factor_model(save=True)
fac_ret, eps, cov = out["factor_returns"], out["residuals"], out["factor_cov"]


[TORANIKO_INPUTS_US] Using 31,529 rows with complete style data and 7 style factors: ['SIZE', 'VAL', 'MOM', 'QUAL', 'EMA20_GAP', 'VOL_Z', 'LIQ']
[TORANIKO_INPUTS_US] Sector columns used: ['sector_Communication Services', 'sector_Consumer Discretionary', 'sector_Consumer Staples', 'sector_Energy', 'sector_Financials', 'sector_Health Care', 'sector_Industrials', 'sector_Information Technology', 'sector_Materials']
[FACTOR_MODEL_US] Saved factor returns (769 rows) to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_returns_us.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_returns_us.csv
[FACTOR_MODEL_US] Saved residuals (769 rows) to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_residuals_us.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_residuals_us.csv
[FACTOR_MODEL_US] Saved factor covariance matrix to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_cov_us.parquet and C:\Users\quantbase\Desktop\SyStrat_fa

In [21]:
fac_ret_us, eps_us, factor_cov_us = estimate_us_factor_model(
    model_start=START,
    model_end=END,
    tickers=US_TICKERS,
    save=True,
)


[TORANIKO_INPUTS_US] Using 31,529 rows with complete style data and 7 style factors: ['SIZE', 'VAL', 'MOM', 'QUAL', 'EMA20_GAP', 'VOL_Z', 'LIQ']
[TORANIKO_INPUTS_US] Sector columns used: ['sector_Communication Services', 'sector_Consumer Discretionary', 'sector_Consumer Staples', 'sector_Energy', 'sector_Financials', 'sector_Health Care', 'sector_Industrials', 'sector_Information Technology', 'sector_Materials']
[FACTOR_MODEL_US] Saved factor returns (769 rows) to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_returns_us.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_returns_us.csv
[FACTOR_MODEL_US] Saved residuals (769 rows) to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_residuals_us.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_residuals_us.csv
[FACTOR_MODEL_US] Saved factor covariance matrix to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\factor_cov_us.parquet and C:\Users\quantbase\Desktop\SyStrat_fa

In [56]:
fac = pd.read_parquet(DATA_INT / "factor_returns_us.parquet")
res = pd.read_parquet(DATA_INT / "factor_residuals_us.parquet")
cov = pd.read_parquet(DATA_INT / "factor_cov_us.parquet")

In [28]:
fac.head()

,market,sector_Communication Services,sector_Consumer Discretionary,sector_Consumer Staples,sector_Energy,sector_Financials,sector_Health Care,sector_Industrials,sector_Information Technology,sector_Materials,EMA20_GAP,LIQ,MOM,QUAL,SIZE,VAL,VOL_Z
date,,,,,,,,,,,,,,,,,
2021-11-02,0.001730,0.001861,-0.006513,0.003607,-0.011131,-0.014072,0.005852,-0.000519,0.008403,0.012513,0.004956,0.001394,0.001188,0.004257,0.000501,-0.002307,-0.006729
2021-11-03,0.004885,0.006107,0.011467,0.001578,-0.009858,-0.006224,0.002056,-0.004620,0.001815,-0.002321,0.003788,0.000185,-0.004937,-0.002133,0.002700,0.000287,0.002906
2021-11-04,0.007573,0.001382,0.009735,0.000330,-0.001522,0.002628,-0.003737,-0.011362,0.000710,0.001836,0.006075,0.001545,-0.007080,-0.000148,0.001171,-0.001204,0.003309
2021-11-05,0.005470,-0.002796,-0.004071,-0.004284,0.004646,0.017368,-0.016948,0.013672,-0.003595,-0.003992,0.001548,0.002584,-0.001303,-0.001083,0.000277,0.005134,-0.001106
2021-11-08,0.003165,-0.005304,-0.011729,-0.013222,0.004207,0.007482,0.001224,0.001348,0.002472,0.013522,0.005164,-0.002133,-0.005546,-0.000277,-0.000367,0.003518,0.005161


In [29]:
fac.columns

Index(['market', 'sector_Communication Services',
       'sector_Consumer Discretionary', 'sector_Consumer Staples',
       'sector_Energy', 'sector_Financials', 'sector_Health Care',
       'sector_Industrials', 'sector_Information Technology',
       'sector_Materials', 'EMA20_GAP', 'LIQ', 'MOM', 'QUAL', 'SIZE', 'VAL',
       'VOL_Z'],
      dtype='object')

In [30]:
fac.describe().T

,count,mean,std,min,25%,50%,75%,max
market,769.0,0.000631,0.010248,-0.041246,-0.004548,0.000447,0.005992,0.048948
sector_Communication Services,769.0,0.000082,0.008221,-0.030007,-0.005304,0.000540,0.005196,0.039662
sector_Consumer Discretionary,769.0,-0.000341,0.007701,-0.027488,-0.005106,-0.000435,0.004703,0.027811
sector_Consumer Staples,769.0,-0.000055,0.007401,-0.029029,-0.004558,-0.000054,0.003986,0.034834
sector_Energy,769.0,0.000220,0.010722,-0.035035,-0.007814,-0.000394,0.007914,0.034529
sector_Financials,769.0,-0.000049,0.007409,-0.025135,-0.004195,0.000131,0.004246,0.025687
sector_Health Care,769.0,-0.000133,0.006433,-0.020434,-0.003982,-0.000516,0.003558,0.023977
sector_Industrials,769.0,0.000241,0.006824,-0.031472,-0.003866,0.000207,0.004831,0.021021
sector_Information Technology,769.0,-0.000004,0.007709,-0.029852,-0.004864,0.000191,0.004788,0.033216
sector_Materials,769.0,0.000039,0.007876,-0.028269,-0.004657,0.000247,0.004451,0.024642


In [31]:
res.head()


,AAPL,ABBV,ABT,AMAT,AMD,AMZN,AVGO,CAT,COST,CRM,...,PG,PLTR,RTX,TMO,TMUS,TSLA,UNH,V,WMT,XOM
date,,,,,,,,,,,,,,,,,,,,,
2021-11-02,-0.006431,-0.001015,-0.021667,0.008326,0.008354,0.005403,-0.001319,0.015664,0.006757,-0.011137,...,-0.002068,-0.009491,-0.000812,-0.005403,0.000222,-0.006897,-0.008097,0.012937,-0.010142,-0.001554
2021-11-03,-0.005051,0.001948,-0.000927,0.010386,0.004439,-0.004755,0.004792,-0.006868,-0.000872,-0.010782,...,-0.001101,0.009628,0.015406,-0.002225,0.010074,-0.011607,-0.001380,-0.002739,-0.008496,0.006686
2021-11-04,-0.017521,-0.012423,-0.003681,0.020891,-0.003395,-0.004249,0.001646,0.005952,0.013694,0.000652,...,-0.006916,0.010412,0.011789,-0.011525,-0.006524,-0.019582,-0.008887,-0.004371,-0.005298,0.010368
2021-11-05,-0.004172,0.011541,-0.011295,0.020160,-0.012435,0.006548,0.014140,-0.008710,-0.004609,-0.000225,...,0.000751,-0.005648,0.000024,-0.008670,-0.007044,-0.005239,0.007766,0.001889,-0.013787,-0.003583
2021-11-08,-0.004443,-0.019108,0.013664,-0.012593,-0.002504,0.002205,-0.005466,0.007514,-0.005539,-0.002137,...,0.004835,0.016178,-0.005950,0.001956,-0.022344,-0.005864,0.004542,0.007225,0.006297,0.007545


In [32]:
res.columns

Index(['AAPL', 'ABBV', 'ABT', 'AMAT', 'AMD', 'AMZN', 'AVGO', 'CAT', 'COST',
       'CRM', 'CSCO', 'CVX', 'GE', 'GOOG', 'HD', 'IBM', 'ISRG', 'JNJ', 'KO',
       'LIN', 'LLY', 'MA', 'MCD', 'META', 'MRK', 'MSFT', 'MU', 'NFLX', 'NVDA',
       'ORCL', 'PEP', 'PG', 'PLTR', 'RTX', 'TMO', 'TMUS', 'TSLA', 'UNH', 'V',
       'WMT', 'XOM'],
      dtype='object')

In [33]:
res.describe().T

,count,mean,std,min,25%,50%,75%,max
AAPL,769.0,-0.000723,0.014606,-0.066325,-0.009727,-0.000633,0.007719,0.050208
ABBV,769.0,0.000340,0.007931,-0.025281,-0.004755,0.000179,0.005625,0.031796
ABT,769.0,0.000226,0.009410,-0.028201,-0.005293,-0.000014,0.005872,0.036504
AMAT,769.0,0.000350,0.009724,-0.029194,-0.006402,0.000092,0.006700,0.052220
AMD,769.0,0.000370,0.010014,-0.039911,-0.005559,0.000105,0.006475,0.050947
AMZN,769.0,-0.000107,0.011454,-0.039210,-0.007085,-0.000109,0.006941,0.043425
AVGO,769.0,-0.000705,0.008812,-0.029343,-0.006214,-0.000673,0.004641,0.036650
CAT,769.0,-0.000256,0.009374,-0.031467,-0.006148,-0.000209,0.005486,0.034421
COST,769.0,-0.000012,0.008895,-0.032882,-0.005612,-0.000137,0.005483,0.038326
CRM,769.0,0.000389,0.010550,-0.038776,-0.006457,0.000300,0.006882,0.048543


In [34]:
cov

,market,sector_Communication Services,sector_Consumer Discretionary,sector_Consumer Staples,sector_Energy,sector_Financials,sector_Health Care,sector_Industrials,sector_Information Technology,sector_Materials,EMA20_GAP,LIQ,MOM,QUAL,SIZE,VAL,VOL_Z
market,0.026463,0.008414,0.008487,-0.010900,-0.006162,0.001840,-0.008899,-0.001306,0.009428,-0.000903,-0.000682,-0.002970,0.001588,-0.000051,0.012927,0.000020,0.010845
sector_Communication Services,0.008414,0.017032,0.008005,-0.006311,-0.011110,-0.001625,-0.005409,-0.005511,0.009553,-0.004623,-0.001286,-0.001674,0.000081,0.000564,0.006940,-0.000693,0.006388
sector_Consumer Discretionary,0.008487,0.008005,0.014943,-0.005282,-0.009017,-0.002296,-0.005830,-0.003897,0.008678,-0.005303,-0.000896,-0.000335,-0.000393,-0.000558,0.005760,-0.000930,0.006528
sector_Consumer Staples,-0.010900,-0.006311,-0.005282,0.013803,0.000168,-0.001979,0.007202,-0.001327,-0.007071,0.000797,0.000286,0.001683,-0.001047,-0.000172,-0.006198,-0.000764,-0.007791
sector_Energy,-0.006162,-0.011110,-0.009017,0.000168,0.028973,-0.003319,0.000771,0.005847,-0.010244,-0.002069,0.001148,0.001510,0.001161,-0.000726,-0.005122,0.004336,-0.002868
sector_Financials,0.001840,-0.001625,-0.002296,-0.001979,-0.003319,0.013833,-0.001156,-0.001067,-0.001986,-0.000404,0.000544,-0.000837,-0.000710,0.000314,0.000697,-0.001165,0.000178
sector_Health Care,-0.008899,-0.005409,-0.005830,0.007202,0.000771,-0.001156,0.010429,-0.000976,-0.005953,0.000922,0.000513,0.000282,-0.000328,0.000627,-0.004763,-0.000888,-0.005522
sector_Industrials,-0.001306,-0.005511,-0.003897,-0.001327,0.005847,-0.001067,-0.000976,0.011735,-0.003902,-0.000902,0.000131,0.000276,0.000610,-0.000670,-0.002246,0.001381,-0.001545
sector_Information Technology,0.009428,0.009553,0.008678,-0.007071,-0.010244,-0.001986,-0.005953,-0.003902,0.014976,-0.004050,-0.000444,-0.000302,0.001067,0.000245,0.006387,-0.001150,0.006636
sector_Materials,-0.000903,-0.004623,-0.005303,0.000797,-0.002069,-0.000404,0.000922,-0.000902,-0.004050,0.015633,0.000004,-0.000602,-0.000442,0.000376,-0.001455,-0.000127,-0.002004


In [37]:
cov.equals(cov.T)      # check - should be True or very close

True

In [38]:
cov.index, cov.columns # same labels

(Index(['market', 'sector_Communication Services',
        'sector_Consumer Discretionary', 'sector_Consumer Staples',
        'sector_Energy', 'sector_Financials', 'sector_Health Care',
        'sector_Industrials', 'sector_Information Technology',
        'sector_Materials', 'EMA20_GAP', 'LIQ', 'MOM', 'QUAL', 'SIZE', 'VAL',
        'VOL_Z'],
       dtype='object'),
 Index(['market', 'sector_Communication Services',
        'sector_Consumer Discretionary', 'sector_Consumer Staples',
        'sector_Energy', 'sector_Financials', 'sector_Health Care',
        'sector_Industrials', 'sector_Information Technology',
        'sector_Materials', 'EMA20_GAP', 'LIQ', 'MOM', 'QUAL', 'SIZE', 'VAL',
        'VOL_Z'],
       dtype='object'))

In [ ]:
diag = cov.values.diagonal()
vol = np.sqrt(diag)    # annual implied factor vols
pd.Series(vol, index=cov.index)

market                           0.162676
sector_Communication Services    0.130505
sector_Consumer Discretionary    0.122243
sector_Consumer Staples          0.117488
sector_Energy                    0.170214
sector_Financials                0.117612
sector_Health Care               0.102125
sector_Industrials               0.108330
sector_Information Technology    0.122377
sector_Materials                 0.125032
EMA20_GAP                        0.068326
LIQ                              0.045368
MOM                              0.051457
QUAL                             0.031061
SIZE                             0.088548
VAL                              0.042912
VOL_Z                            0.095935
dtype: float64

In [41]:
std = np.sqrt(diag)
corr = cov / std[:, None] / std[None, :]     #turn covariance into correlation
corr


,market,sector_Communication Services,sector_Consumer Discretionary,sector_Consumer Staples,sector_Energy,sector_Financials,sector_Health Care,sector_Industrials,sector_Information Technology,sector_Materials,EMA20_GAP,LIQ,MOM,QUAL,SIZE,VAL,VOL_Z
market,1.000000,0.396326,0.426796,-0.570304,-0.222521,0.096168,-0.535671,-0.074105,0.473588,-0.044380,-0.061380,-0.402461,0.189726,-0.010147,0.897426,0.002927,0.694922
sector_Communication Services,0.396326,1.000000,0.501758,-0.411614,-0.500127,-0.105897,-0.405876,-0.389785,0.598140,-0.283298,-0.144166,-0.282717,0.012007,0.139197,0.600574,-0.123702,0.510183
sector_Consumer Discretionary,0.426796,0.501758,1.000000,-0.367801,-0.433358,-0.159720,-0.467021,-0.294274,0.580122,-0.346985,-0.107273,-0.060437,-0.062406,-0.146995,0.532100,-0.177363,0.556672
sector_Consumer Staples,-0.570304,-0.411614,-0.367801,1.000000,0.008407,-0.143185,0.600209,-0.104273,-0.491805,0.054271,0.035622,0.315727,-0.173155,-0.047079,-0.595818,-0.151581,-0.691265
sector_Energy,-0.222521,-0.500127,-0.433358,0.008407,1.000000,-0.165805,0.044382,0.317073,-0.491788,-0.097214,0.098706,0.195490,0.132595,-0.137272,-0.339841,0.593575,-0.175622
sector_Financials,0.096168,-0.105897,-0.159720,-0.143185,-0.165805,1.000000,-0.096234,-0.083739,-0.138007,-0.027471,0.067708,-0.156814,-0.117272,0.085864,0.066967,-0.230883,0.015798
sector_Health Care,-0.535671,-0.405876,-0.467021,0.600209,0.044382,-0.096234,1.000000,-0.088179,-0.476325,0.072177,0.073481,0.060840,-0.062443,0.197697,-0.526673,-0.202546,-0.563583
sector_Industrials,-0.074105,-0.389785,-0.294274,-0.104273,0.317073,-0.083739,-0.088179,1.000000,-0.294362,-0.066629,0.017643,0.056162,0.109461,-0.199167,-0.234118,0.297107,-0.148664
sector_Information Technology,0.473588,0.598140,0.580122,-0.491805,-0.491788,-0.138007,-0.476325,-0.294362,1.000000,-0.264718,-0.053091,-0.054440,0.169507,0.064523,0.589417,-0.218986,0.565212
sector_Materials,-0.044380,-0.283298,-0.346985,0.054271,-0.097214,-0.027471,0.072177,-0.066629,-0.264718,1.000000,0.000478,-0.106187,-0.068753,0.096705,-0.131447,-0.023578,-0.167065


In [ ]:
#---------Decomp

In [48]:
# factor panel (no sectors needed; helper will attach them inside decompose)
factor_panel = _load_factor_panel_us()  # funds_panel + style factors

# factor model outputs
factor_returns, residuals = _load_factor_model_us()

In [52]:
tickers_model = list(residuals.columns)   # e.g. AAPL, ABBV, ...
dates_model   = factor_returns.index      # e.g. 2021-11-02 ... 2021-XX-YY

weights_eq = build_equal_weight_portfolio(
    tickers=tickers_model,
    dates=dates_model,
)

In [53]:
# 2) Attach sector exposures so columns match the sector_* factors
factor_panel_with_sectors = _attach_sector_exposures_us(
    panel=factor_panel,
    factor_returns=factor_returns,
)

factor_panel_with_sectors.to_csv(DATA_INT / "factor_panel_us_with_sectors.csv")
factor_panel_with_sectors.to_parquet(DATA_INT / "factor_panel_us_with_sectors.parquet")

In [54]:
# 3) Quick sanity check
factor_panel_with_sectors.head()

,date,ticker,market,ret,mkt_cap,SIZE,VAL,MOM,QUAL,ROE_z,...,LIQ,sector_Communication Services,sector_Consumer Discretionary,sector_Consumer Staples,sector_Energy,sector_Financials,sector_Health Care,sector_Industrials,sector_Information Technology,sector_Materials
0,2020-11-03,AAPL,US,0.015353,1.877679e+12,2.519633,-0.458390,NaN,-0.419834,0.345682,...,1.959010,False,False,False,False,False,False,False,True,False
1,2020-11-03,ABBV,US,-0.003286,1.552911e+11,-0.214317,0.767764,NaN,1.665262,0.037950,...,-0.136873,False,False,False,False,False,True,False,False,False
2,2020-11-03,ABT,US,0.019352,1.941799e+11,0.030817,-0.570250,NaN,-0.355978,-0.254101,...,-0.788841,False,False,False,False,False,True,False,False,False
3,2020-11-03,AMAT,US,0.024863,5.651262e+10,-1.323079,0.695128,NaN,0.002327,-0.071370,...,-0.007083,False,False,False,False,False,False,False,True,False
4,2020-11-03,AMD,US,0.025167,9.210366e+10,-0.787310,-1.237910,NaN,-1.195322,-0.160618,...,1.166198,False,False,False,False,False,False,False,True,False


In [55]:
factor_contrib, resid_contrib, total_model = decompose_portfolio_us(
    weights=weights_eq,
    factor_panel=factor_panel_with_sectors,  # enriched panel
    factor_returns=factor_returns,
    residuals=residuals,
)

In [56]:
true_ret, model_ret, diff, rmse, max_abs_diff = sanity_check_decomposition_us(
    weights=weights_eq,
    factor_panel=factor_panel,
    total_from_model=total_model,
)

In [57]:
rmse, max_abs_diff, pd.concat(
    [true_ret, model_ret, diff.rename("diff")], axis=1
).head()

(0.0014236578066900685,
 0.00828486386478378,
             portfolio_return_true  total_from_model      diff
 date                                                         
 2021-11-02               0.003650          0.003837 -0.000187
 2021-11-03               0.007585          0.006681  0.000904
 2021-11-04               0.010575          0.007705  0.002870
 2021-11-05               0.000275          0.001768 -0.001493
 2021-11-08               0.004338          0.002623  0.001715)

In [74]:
# Diagnostic

In [75]:
# Quick diagnostic in a scratch cell or notebook
import pandas as pd
from pathlib import Path

DATA_INT = Path(r"C:\Users\quantbase\Desktop\SyStrat_factmod\data_int")

mkt = pd.read_parquet(DATA_INT / "mkt_cap_all.parquet")
print("mkt_cap_all tickers:", mkt["ticker"].unique())
print(mkt.groupby("ticker")["mkt_cap"].apply(lambda s: s.notna().mean()))


mkt_cap_all tickers: ['AAPL' 'MSFT' 'RELIANCE' 'SBIN']
ticker
AAPL        1.0
MSFT        1.0
RELIANCE    0.0
SBIN        0.0
Name: mkt_cap, dtype: float64


In [76]:
import pandas as pd
from pathlib import Path

DATA_INT = Path(r"C:\Users\quantbase\Desktop\SyStrat_factmod\data_int")
px = pd.read_parquet(DATA_INT / "prices_us.parquet")
print(px["ticker"].unique())
print(px["ticker"].value_counts())


['AAPL' 'AMZN' 'META' 'MSFT' 'NVDA']
ticker
AAPL    1232
AMZN    1232
META    1232
MSFT    1232
NVDA    1232
Name: count, dtype: int64


In [78]:
import pandas as pd
from src.fm_config import DATA_INT

# 1) What’s in returns_all?
ret = pd.read_parquet(DATA_INT / "returns_all.parquet")
print("returns_all US tickers:", ret[ret["market"] == "US"]["ticker"].unique())

returns_all US tickers: ['AAPL' 'AMZN' 'META' 'MSFT' 'NVDA']


In [81]:
# 2) What’s in mkt_cap_all right now?
mkt = pd.read_parquet(DATA_INT / "mkt_cap_all.parquet")
print("mkt_cap_all tickers:", mkt["ticker"].unique())
print(
    mkt[mkt["market"] == "US"]
    .groupby("ticker")["mkt_cap"]
    .apply(lambda s: s.gt(0).mean())
)


mkt_cap_all tickers: ['AAPL' 'MSFT' 'RELIANCE' 'SBIN']
ticker
AAPL    1.0
MSFT    1.0
Name: mkt_cap, dtype: float64


In [13]:
import subprocess, sys
from pathlib import Path

PROJ_ROOT = Path(r"C:\Users\quantbase\Desktop\SyStrat_factmod")

def run_cli_module(mod_name: str):
    """
    Run `python -m <mod_name>` with the project root as cwd
    and print stdout/stderr.
    """
    print(f"\n=== Running: python -m {mod_name} ===")
    result = subprocess.run(
        [sys.executable, "-m", mod_name],
        cwd=PROJ_ROOT,
        text=True,
        capture_output=True,
    )
    print(result.stdout)
    if result.stderr:
        print("STDERR:\n", result.stderr)


In [14]:
run_cli_module("src.fm_data_prices")
run_cli_module("src.fm_fundamentals")
run_cli_module("src.fm_returns")
run_cli_module("src.fm_panel_us")
run_cli_module("src.fm_factors")
run_cli_module("src.fm_factor_panel")
run_cli_module("src.fm_factor_model_us")
run_cli_module("src.fm_decompose_us")



=== Running: python -m src.fm_data_prices ===
=== US (SimFin) demo ===
Dataset "us-shareprices-daily" on disk (1 days old).
- Loading from disk ... Done!
[US] Saved 140 rows to C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\prices_us.parquet and C:\Users\quantbase\Desktop\SyStrat_factmod\data_int\prices_us.csv
  ticker       date  SimFinId  ...     volume  Dividend  shares_outstanding
0   AAPL 2019-12-19    111052  ...   98507788       NaN        1.777306e+10
1   AAPL 2019-12-20    111052  ...  276130972       NaN        1.777306e+10
2   AAPL 2019-12-23    111052  ...   98711532       NaN        1.777306e+10
3   AAPL 2019-12-24    111052  ...   48478856       NaN        1.777306e+10
4   AAPL 2019-12-26    111052  ...   93336016       NaN        1.777306e+10

[5 rows x 11 columns]

=== NSE (jugaad-data) demo ===
[NSE] Fetching SBIN from 2020-01-01 to 2020-03-31 (series=EQ)
[NSE] Fetching RELIANCE from 2020-01-01 to 2020-03-31 (series=EQ)
[NSE] Saved 155 rows to C:\Users\quantbase\

In [26]:
panel_us = pd.read_parquet(DATA_INT / "funds_panel_us.parquet")

panel_us[panel_us["inc_report_date"].notna()].head()

,date,ticker,market,ret,mkt_cap,inc_report_date,inc_SimFinId,inc_Currency,inc_Fiscal Year,inc_Fiscal Period,...,cf_Change in Fixed Assets & Intangibles,cf_Net Change in Long Term Investment,cf_Net Cash from Acquisitions & Divestitures,cf_Net Cash from Investing Activities,cf_Dividends Paid,cf_Cash from (Repayment of) Debt,cf_Cash from (Repurchase of) Equity,cf_Net Cash from Financing Activities,cf_Net Change in Cash,inc_age_days
6,2019-12-31,AAPL,US,0.007272,1.287599e+12,2019-12-31,111052.0,USD,2020.0,Q1,...,-9.247000e+09,3.827500e+10,-1.415000e+09,2.638400e+10,-1.409000e+10,-7.594000e+09,-7.802400e+10,-1.027070e+11,-3.106000e+09,0.0
7,2020-01-02,AAPL,US,0.022885,1.317066e+12,2019-12-31,111052.0,USD,2020.0,Q1,...,-9.247000e+09,3.827500e+10,-1.415000e+09,2.638400e+10,-1.409000e+10,-7.594000e+09,-7.802400e+10,-1.027070e+11,-3.106000e+09,2.0
8,2020-01-03,AAPL,US,-0.009722,1.304262e+12,2019-12-31,111052.0,USD,2020.0,Q1,...,-9.247000e+09,3.827500e+10,-1.415000e+09,2.638400e+10,-1.409000e+10,-7.594000e+09,-7.802400e+10,-1.027070e+11,-3.106000e+09,3.0
9,2020-01-06,AAPL,US,0.007934,1.314611e+12,2019-12-31,111052.0,USD,2020.0,Q1,...,-9.247000e+09,3.827500e+10,-1.415000e+09,2.638400e+10,-1.409000e+10,-7.594000e+09,-7.802400e+10,-1.027070e+11,-3.106000e+09,6.0
10,2020-01-07,AAPL,US,-0.004670,1.308472e+12,2019-12-31,111052.0,USD,2020.0,Q1,...,-9.247000e+09,3.827500e+10,-1.415000e+09,2.638400e+10,-1.409000e+10,-7.594000e+09,-7.802400e+10,-1.027070e+11,-3.106000e+09,7.0
